In [1]:
from keras import backend as K
import tensorflow as tf
from math import ceil
import numpy as np
import math
from ssd_encoder_decoder.matching_utils import match_bipartite_greedy32
from bounding_box_utils.bounding_box_utils import iou_float, convert_coordinates
import timeit

Using TensorFlow backend.


In [2]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
K.set_session(sess)
K.clear_session()

def eva(x):
    return(K.eval(x))

In [3]:
def tf_intersection_area_(boxes1, boxes2, coords='corners', mode='outer_product', border_pixels='half'):
    m = tf.shape(boxes1)[0]
    n = tf.shape(boxes2)[0]
    
    xmin = 0
    ymin = 1
    xmax = 2
    ymax = 3

    d = 0
    
    box1xminxmax = tf.slice(boxes1, [0, 0], [m, xmax])
    box2xminxmax = tf.slice(boxes2, [0, 0], [n, xmax])                      
                            
    min_xy = tf.maximum(tf.tile(tf.expand_dims(box1xminxmax,1),[1, n, 1]),
                        tf.tile(tf.expand_dims(box2xminxmax,0),[m, 1, 1]))
    
    box1xmaxymax = tf.slice(boxes1, [0, 2], [m, 2])
    box2xmaxymax = tf.slice(boxes2, [0, 2], [n, 2])                      

    max_xy = tf.minimum(tf.tile(tf.expand_dims(box1xmaxymax,1),[1, n, 1]),
                    tf.tile(tf.expand_dims(box2xmaxymax,0),[m, 1, 1]))

    side_lengths = tf.maximum(tf.cast(0, dtype=tf.int64), tf.add(tf.subtract(max_xy,min_xy),d))

    return tf.multiply(side_lengths[:,:,0],side_lengths[:,:,1])

In [4]:
def tf_iou(boxes1, boxes2, coords='centroids', mode='outer_product', border_pixels='half'):

    intersection_areas = tf_intersection_area_(boxes1, boxes2, coords=coords, mode=mode)
    m = tf.shape(boxes1)[0]
    n = tf.shape(boxes2)[0]
    
    xmin = 0
    ymin = 1
    xmax = 2
    ymax = 3

    d = 0
    
    boxes1_areas = tf.tile(tf.expand_dims(tf.multiply(tf.add(tf.subtract(boxes1[:,xmax],boxes1[:,xmin]),d),tf.add(tf.subtract(boxes1[:,ymax],boxes1[:,ymin]),d)), 1), [1,n])
    boxes2_areas = tf.tile(tf.expand_dims(tf.multiply(tf.add(tf.subtract(boxes2[:,xmax],boxes2[:,xmin]),d),tf.add(tf.subtract(boxes2[:,ymax],boxes2[:,ymin]),d)), 0), [m,1])
    
    union_areas = tf.subtract(tf.add(boxes1_areas,boxes2_areas),intersection_areas)
    op = tf.divide(intersection_areas,union_areas)
    return op

In [5]:
y = np.load('y_pred.npy')
x = np.load('y_true.npy')

y = tf.convert_to_tensor(y)
x = tf.convert_to_tensor(x)

t = tf_iou(x,y)

In [143]:
def match_bipartite_greedy_tf(weight_matrix):
    weight_matrix = tf.identity(weight_matrix)
    num_ground_truth_boxes = tf.cast(tf.shape(weight_matrix)[0], dtype=tf.int32)
    all_gt_indices = tf.range(num_ground_truth_boxes)
    matches = tf.zeros([num_ground_truth_boxes], tf.int64)
    
    print("matches:", K.eval(matches).shape)
    
    #for loop here
    anchor_indices = tf.argmax(weight_matrix, axis=1)
    anchor_indices = tf.cast(anchor_indices,dtype=tf.int32)
    c = tf.stack([all_gt_indices, anchor_indices], axis=1)
    overlaps = tf.gather_nd(weight_matrix,c)
    ground_truth_index = tf.argmax(overlaps)
    anchor_index = anchor_indices[ground_truth_index]

    matches = tf.scatter_update(matches,[ground_truth_index],anchor_index)
    print("matches:", K.eval(matches).shape)
    
    matches[ground_truth_index] = [anchor_index]
    
    weight_matrix[ground_truth_index] = 0
    weight_matrix[:,anchor_index] = 0
    return matches
    

In [144]:
match_bipartite_greedy_tf(t)

matches: (50,)


TypeError: 'ScatterUpdate' Op requires that input 'ref' be a mutable tensor (e.g.: a tf.Variable)

In [133]:
v = tf.Variable(tf.zeros([5], tf.int64))

In [134]:
v =  tf.zeros([5], tf.int64) 

In [97]:
tf.scatter_update(v,1,1)

<tf.Tensor 'ScatterUpdate_4:0' shape=(5,) dtype=int64_ref>